# Membership Inference over Diffusion-models-based Synthetic Tabular Data (MIDST) Challenge @ SaTML 2025.

## White Box Single Table Competition
Welcome to the MIDST challenge!

The MIDST challenge is a multi-track competition aiming to quantitatively evaluate the privacy of synthetic tabular data generated by diffusion models, with a specific focus on its resistance to membership inference attacks (MIAs).

This competition focuses on White Box MIA on tabular diffusion models trained on a single table transaction dataset. The schema of the transaction dataset is as follows:
| trans_id | account_id | trans_date | trans_type | operation | amount  | balance  | k_symbol | bank | account |
|----------|------------|------------|------------|-----------|---------|----------|----------|------|---------|
| integer  | integer    | integer    | integer    | integer   | float   | float    | integer  | integer | integer |

MIA will be performed over two state-of-the-art methods [TabSyn](https://arxiv.org/pdf/2310.09656) and [TabDDPM](https://arxiv.org/pdf/2209.15421). A collection of TabSyn and TabDDPM models will be trained on random subsets of the transaction dataset. The goal is to create an approach (MIA) that can distinguish between samples used to train a model (train data) and other data randomly sampled from the transaction dataset (holdout data) given the model and it's output synthetic data. The `final` set includes 20 models, each with its own set of challenge points (ie train and holdout data), to evaluate solutions on. To facilitate designing an attack, 30 `train` models are provided with comprehensive information about the model, training data and output synthetic data. Additionally, 20 `dev` models are provided to assist in evaluating the effectiveness of attacks prior to making a final submission to the `final` set. Participants can choose to perform MIA over one of or both TabSyn and TabDDPM. In the case of both, the attack that obtains the highest score will be used to rank the submission. A high level summary of the competition is below:
![wbox_diagram_final](https://github.com/user-attachments/assets/2ebb5eed-a6e3-433a-8769-4310b7fbc822)

This notebook will walk you through the process of creating and packaging a submission to the white box single table challenge.

## Package Imports and Evironment Setup

Ensure that you have installed the proper dependenices to run the notebook. The environment installation instructions are available [here](https://github.com/VectorInstitute/MIDSTModels/tree/main/starter_kits). Now that we have verfied we have the proper packages installed, lets import them and define global variables:

In [ ]:
import csv
import os
import random
import zipfile

from pathlib import Path
from functools import partial
from typing import Callable, Any

import numpy as np
import torch

from tqdm.notebook import tqdm
from data import get_challenge_points
from metrics import get_tpr_at_fpr



In [2]:
%cd ..

c:\Users\ksush\attacks\MIDSTModels


C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from midst_models.single_table_TabDDPM.complex_pipeline import tabddpm_whitebox_load_pretrained

In [4]:
TABDDPM_DATA_DIR = "tabddpm_white_box"
TABSYN_DATA_DIR = "tabsyn_white_box"

In [5]:

def get_FLAGS():
    def FLAGS(x): return x
    FLAGS.T = 1000
    FLAGS.ch = 128
    FLAGS.ch_mult = [1, 2, 2, 2]
    FLAGS.attn = [1]
    FLAGS.num_res_blocks = 2
    FLAGS.dropout = 0.1
    FLAGS.beta_1 = 0.0001
    FLAGS.beta_T = 0.02

    return FLAGS

FLAGS = get_FLAGS()


In [19]:
import torch
import numpy as np
import logging
import csv
import os
from typing import Dict, Type
from torch.nn.functional import normalize
import midst_models.attack.components as components

class EpsGetter(components.EpsGetter):
    def __call__(self, xt: torch.Tensor, condition: torch.Tensor = None, noise_level=None, t: int = None) -> torch.Tensor:
        # Access the diffusion model from the dictionary structure
        model = self.model[(None, 'trans')]['diffusion']

        t = torch.ones([xt.shape[0]], device=xt.device).long() * t
        return model._denoise_fn(xt, timesteps=t)

ATTACKERS: Dict[str, Type[components.DDIMAttacker]] = {
    "PIA": components.PIA,
    "PIAN": components.PIAN,
}


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


def attack(base_dir, attacker_name="PIA", attack_num=30, interval=10, seed=0):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    # logger.addHandler(RichHandler())

    logger.info("loading the attacked model...")

    # Initialize attacker
    phases = ["train"]
    
    logger.info("attack start...")
    for phase in phases:
        root = os.path.join(base_dir, phase)
        for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
            if model_folder.endswith("30"):
                path = os.path.join(root, model_folder)
                print("path",path)
                model = tabddpm_whitebox_load_pretrained(path)
                attacker = ATTACKERS[attacker_name](
                    torch.from_numpy(np.linspace(FLAGS.beta_1, FLAGS.beta_T, FLAGS.T)).to(DEVICE), interval, attack_num, EpsGetter(model), lp=4)

                # Move the points to GPU
                challenge_points = get_challenge_points(path)

                # Get raw predictions
                raw_predictions = torch.stack([attacker(cp.to(DEVICE).float()) for cp in challenge_points])

                # Convert to binary predictions based on membership
                # First normalize the predictions to [0,1] range
                normalized_preds = []
                for pred_batch in raw_predictions:
                    # Handle each prediction batch separately
                    batch_min = pred_batch.min()
                    batch_max = pred_batch.max()
                    
                    # Avoid division by zero if all predictions are the same
                    if batch_max == batch_min:
                        normalized = torch.zeros_like(pred_batch)
                    else:
                        normalized = (pred_batch - batch_min) / (batch_max - batch_min)
                    
                    # Convert to binary predictions using 0.5 as threshold
                    binary_preds = (normalized >= 0.5).float()
                    normalized_preds.append(binary_preds)
                
                # Stack all normalized predictions
                final_predictions = torch.stack(normalized_preds)
                predictions_cpu = final_predictions.cpu().detach().numpy()
                
                # Verify predictions are binary (0 or 1)
                assert np.all(np.logical_or(predictions_cpu == 0, predictions_cpu == 1)), "Predictions are not binary"
                
                # Write predictions to file
                with open(os.path.join(path, "prediction.csv"), mode="w", newline="") as file:
                    writer = csv.writer(file)
                    # Write each value in a separate row
                    for value in predictions_cpu.squeeze():
                        writer.writerow([value])

                # Optional logging of sample predictions
                print(f"raw_predictions.shape: {raw_predictions.shape}")
                print(f"Sample binary predictions: {predictions_cpu.squeeze()[:5]}")
                

In [20]:
attack(base_dir="tabddpm_white_box",
        attacker_name="PIA",
        attack_num=2,
        interval=10,)

path tabddpm_white_box\train\tabddpm_30
Checkpoint found, loading...


C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\ksush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator QuantileTransformer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


raw_predictions.shape: torch.Size([200, 2, 8])
Sample binary predictions: [[[0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 1. 1. 1.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 1. 1. 1.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 1. 1. 1.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 1. 1. 1.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 1. 1. 1.]]]


## Task

Your task as a competitor is to produce, for each model in `dev` and `final` in `tabddpm_white_box` and `tabsyn_white_box`, a CSV file listing your confidence scores (values between 0 and 1) for the membership of the challenge examples. You must save these scores in a `prediction.csv` file and place it in the same folder as the corresponding model. A submission to the challenge is an an archive containing just these `prediction.csv` files.

**You must submit predictions for both `dev` and `final` when you submit to CodaBench.**

In the following, we will show you how to correctly package a submission to the competition. To focus solely on the submission logic, the attack model will simply generate random predictions. Let's start by creating baseline attack models `tabddpm_attack_model` and `tabsyn_attack_model` based on their respective shadow models:

In [5]:
def get_attack_model(base_train_path: Path) -> Callable[[Any], float]:
    return lambda x : random.uniform(0, 1)

base_tabddpm_train_path = os.path.join(TABDDPM_DATA_DIR, "train")
base_tabsyn_train_path = os.path.join(TABSYN_DATA_DIR, "train")
tabddpm_attack_model = get_attack_model(base_tabddpm_train_path)
tabsyn_attack_model = get_attack_model(base_tabsyn_train_path)

Using the attack model, we can obtain predictions for each point in the challenge point set for train, dev and final:

In [6]:
phases = ["train", "dev", "final"]

for base_dir, attack_model in zip([TABDDPM_DATA_DIR, TABSYN_DATA_DIR], [tabddpm_attack_model, tabsyn_attack_model]):
    for phase in phases:
        root = os.path.join(base_dir, phase)
        for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
            path = os.path.join(root, model_folder)
    
            challenge_points = get_challenge_points(path)
    
            predictions = torch.Tensor([attack_model(cp) for cp in challenge_points])
           
            assert torch.all((0 <= predictions) & (predictions <= 1))
            with open(os.path.join(path, "prediction.csv"), mode="w", newline="") as file:
                writer = csv.writer(file)
    
                # Write each value in a separate row
                for value in list(predictions.numpy().squeeze()):
                    writer.writerow([value])

## Scoring

Let's see how the attack does on `train`, for which we have the ground truth.
When preparing a submission, you can use part of `train` to develop an attack and a held-out part to evaluate your attack.

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
import os
import torch

def get_tpr_at_fpr(y_true, y_score, target_fpr=0.1):
    """Calculate TPR at a specific FPR threshold."""
    # Sort scores and corresponding truth values
    desc_score_indices = np.argsort(y_score, kind="mergesort")[::-1]
    y_score = y_score[desc_score_indices]
    y_true = y_true[desc_score_indices]
    
    # Calculate cumulative FPR and TPR
    n_neg = np.sum(y_true == 0)
    n_pos = np.sum(y_true == 1)
    tpr = np.cumsum(y_true) / n_pos
    fpr = np.cumsum(1 - y_true) / n_neg
    
    # Find TPR at target FPR
    for i in range(len(fpr)):
        if fpr[i] >= target_fpr:
            return tpr[i]
    return 1.0

def evaluate_predictions(base_dirs, num_attackers=2):
    tpr_at_fpr_list = []
    auc_list = []
    
    for base_dir in base_dirs:
        predictions = []
        solutions = []
        root = os.path.join(base_dir, "train")
        
        for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
            path = os.path.join(root, model_folder)
            
            # Load predictions - reshape to account for multiple attackers
            pred = np.loadtxt(os.path.join(path, "prediction.csv"))
            pred = pred.reshape(-1, num_attackers, 8)  # [N_points, num_attackers, 8]
            
            # Average predictions across attackers
            pred_mean = np.mean(pred, axis=1)  # [N_points, 8]
            predictions.append(pred_mean)
            
            # Load ground truth
            solution = np.loadtxt(os.path.join(path, "challenge_label.csv"), skiprows=1)
            solutions.append(solution)
        
        predictions = np.concatenate(predictions)  # Shape: [N_total_points, 8]
        solutions = np.concatenate(solutions)      # Shape: [N_total_points]
        
        # Calculate metrics for each dimension
        for dim in range(predictions.shape[1]):
            dim_preds = predictions[:, dim]
            
            # Calculate TPR at FPR
            tpr_at_fpr = get_tpr_at_fpr(solutions, dim_preds)
            tpr_at_fpr_list.append(tpr_at_fpr)
            
            # Calculate AUC
            auc = roc_auc_score(solutions, dim_preds)
            auc_list.append(auc)
            
            print(f"{os.path.basename(base_dir)} Train Attack Dimension {dim}:")
            print(f"  TPR at FPR==10%: {tpr_at_fpr:.4f}")
            print(f"  AUC: {auc:.4f}")
    
    # Get best scores across all dimensions
    final_tpr_at_fpr = max(tpr_at_fpr_list)
    final_auc = max(auc_list)
    
    print(f"\nBest scores across dimensions:")
    print(f"Final Train Attack TPR at FPR==10%: {final_tpr_at_fpr:.4f}")
    print(f"Final Train Attack AUC: {final_auc:.4f}")
    
    return final_tpr_at_fpr, final_auc

# Example usage
base_dirs = [TABDDPM_DATA_DIR]  # Add TABSYN_DATA_DIR if needed
final_tpr, final_auc = evaluate_predictions(base_dirs, num_attackers=2)

ValueError: could not convert string '"[[0.' to float64 at row 0, column 1.

## Packaging the submission

Now we can store the predictions into a zip file, which you can submit to CodaBench. Importantly, we create a single zip file for dev and final. The structure of the submission is as follows:

```
└── root_folder
    ├── tabsyn_white_box
    │   ├── dev
    │   │   └── tabsyn_#
    │   │       └── prediction.csv
    │   └── final
    │       └── tabsyn_#
    │           └── prediction.csv
    └── tabddpm_white_box
        ├── dev 
        │   └── tabddpm_#
        │       └── prediction.csv
        └── final 
            └── tabddpm_# 
                └── prediction.csv
```
**Note:** The `root_folder` can have any name but it is important all of the subdirectories follow the above structure and naming conventions. 

If a participant is looking to submit an attack for only one of TabSyn and TabDDPM, they can simply omit the other directory (ie `tabddpm_white_box` or `tabsyn_white_box` from the root_folder).

In [8]:
with zipfile.ZipFile(f"white_box_single_table_submission.zip", 'w') as zipf:
    for phase in ["dev", "final"]:
        for base_dir in [TABDDPM_DATA_DIR, TABSYN_DATA_DIR]:
            root = os.path.join(base_dir, phase)
            for model_folder in sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])):
                path = os.path.join(root, model_folder)
                if not os.path.isdir(path): continue

                file = os.path.join(path, "prediction.csv")
                if os.path.exists(file):
                    # Use `arcname` to remove the base directory and phase directory from the zip path
                    arcname = os.path.relpath(file, os.path.dirname(base_dir))
                    zipf.write(file, arcname=arcname)
                else:
                    raise FileNotFoundError(f"`prediction.csv` not found in {path}.")

The generated white_box_single_table_submission.zip can be directly submitted to the dev phase in the CodaBench UI. Although this submission contains your predictions for both the dev and final set, you will only receive feedback on your predictions for the dev phase. The predictions for the final phase will be evaluated once the competiton ends using the most recent submission to the dev phase.